In [1]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import numpy as np
from pandas import read_excel

In [2]:
df = pd.read_csv('./assignments_from_pool_pedestrian.tsv', sep="\t")


In [3]:
df = df.drop(columns=['GOLDEN:result', 'HINT:text', 'HINT:default_language'])

In [4]:
# Словарь для значений правильных ответов, меняем все кроме 'no_stop' на "no"
d = {'pedestrian':'pedestrian','main_road':'no','no_stop':'no','bus_stop':'no','give_way':'no'}
# Заменяем правльные ответы на 'no' и 'give_way'
df['label'] = df['INPUT:label'].map(d)

In [5]:
# Считаем TP, TN, FP, FN (по всем толокерам)
(df['label'] +'_'+ df['OUTPUT:result']).value_counts()

no_no                    599
pedestrian_pedestrian    147
pedestrian_no              3
no_pedestrian              1
dtype: int64

In [6]:
TP = df[(df['label'] == 'pedestrian') & (df['OUTPUT:result'] == 'pedestrian')].count()[0]
TN = df[(df['label'] == 'no') & (df['OUTPUT:result'] == 'no')].count()[0]
FN = df[(df['label'] == 'pedestrian') & (df['OUTPUT:result'] == 'no')].count()[0]
FP = df[(df['label'] == 'no') & (df['OUTPUT:result'] == 'pedestrian')].count()[0]
print('TP', TP)
print('TN', TN)
print('FP', FP)
print('FN', FN)

TP 147
TN 599
FP 1
FN 3


In [7]:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
Accuracy
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
ROC_AUC = (1 + TPR - FPR)/2
print("Accuracy", Accuracy)
print('TPR', TPR)
print('FPR', FPR)
print('ROC_AUC', ROC_AUC)

Accuracy 0.9946666666666667
TPR 0.98
FPR 0.0016666666666666668
ROC_AUC 0.9891666666666666


In [8]:
#df.groupby('INPUT:image')

In [9]:
#Считаем результат по большинству ответов толокеров по каждой задаче, записываем в словарь w. Задача 'INPUT:image' и 'OUTPUT:result'
w = {}
N = 0
for i in df['INPUT:image'].unique():
    N += 1
    #print(N, i)
    #display(df[df['INPUT:image']== i]['OUTPUT:result'].value_counts())
    w[i] = df[df['INPUT:image']== i]['OUTPUT:result'].value_counts().index[0]
    #if df[df['INPUT:image']== i]['OUTPUT:result'].value_counts()[0] != 3:
    #    break

In [10]:
#w


In [11]:
# Создаем df2 без перекрытия 
df2 = df[['INPUT:image','INPUT:label', 'label']].drop_duplicates()

In [12]:
# в df2 создаем столбец с разметкой толокеров, по большинству из словаря w
df2['tolokers'] =df2['INPUT:image'].map(w)


In [13]:
# Считаем TP, TN, FP, FN (по всем уникальным задачам, используя большинство) 
(df2['label'] +' - '+ df2['tolokers']).value_counts()

no - no                    200
pedestrian - pedestrian     50
dtype: int64

In [14]:

TP = df2[(df2['label'] == 'pedestrian') & (df2['tolokers'] == 'pedestrian')].count()[0]
TN = df2[(df2['label'] == 'no') & (df2['tolokers'] == 'no')].count()[0]
FN = df2[(df2['label'] == 'pedestrian') & (df2['tolokers'] == 'no')].count()[0]
FP = df2[(df2['label'] == 'no') & (df2['tolokers'] == 'pedestrian')].count()[0]
print('TP', TP)
print('TN', TN)
print('FP', FP)
print('FN', FN)

TP 50
TN 200
FP 0
FN 0


In [15]:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
Accuracy
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
ROC_AUC = (1 + TPR - FPR)/2
print("Accuracy", Accuracy)
print('TPR', TPR)
print('FPR', FPR)
print('ROC_AUC', ROC_AUC)

Accuracy 1.0
TPR 1.0
FPR 0.0
ROC_AUC 1.0
